In [5]:
# carregar base de dados
from test_helper import Test
import os.path
baseDir = os.path.join('Data')
inputPath = os.path.join('Aula04', 'BikeSharing.csv')
fileName = os.path.join(baseDir, inputPath)

numPartitions = 2
csvData = sc.textFile(fileName, numPartitions)
header = csvData.take(1)[0]
rawData = csvData.filter(lambda x: x!=header)

'''
https://www.kaggle.com/c/bike-sharing-demand
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals
'''

''

In [3]:
print header
print
print rawData.take(1)

datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

[u'2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,3,13,16']


In [105]:
# alterar essa célula invalida o projeto!!!
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import DenseVector
from datetime import datetime as dt
import numpy as np

def parsePoint(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    data = point.split(',')
    Date = dt.strptime(data[0], "%Y-%m-%d %H:%M:%S")
    DateList = [Date.year-2011, Date.month/12.0, Date.hour/24.0]
    
    season = [0]*4
    season[int(data[1])-1] = 1
    
    dividends = [4., 41., 46., 100., 57.]
    realValues =  map(lambda x: float(x[0])/x[1], zip(data[2:-3], dividends))
    
    features = DenseVector(DateList + season + realValues)
    
    label = int(data[-1])
    
    return LabeledPoint(label, features)

weights = [.8, .1, .1]
seed = 42

rawTrainData, rawValData, rawTestData = rawData.randomSplit(weights, seed)

parsedTrainData = rawTrainData.map(parsePoint).cache()
parsedValData = rawValData.map(parsePoint).cache()
parsedTestData = rawTestData.map(parsePoint).cache()

print parsedTrainData.count()

print parsedTrainData.map(lambda x: x.features[-1]).min()
print parsedTrainData.map(lambda x: x.features[-2]).min()
print parsedTrainData.map(lambda x: x.features[-3]).min()
print parsedTrainData.map(lambda x: x.features[-4]).min()
print parsedTrainData.map(lambda x: x.features[-5]).min()
print
print parsedTrainData.map(lambda x: x.features[-1]).max()
print parsedTrainData.map(lambda x: x.features[-2]).max()
print parsedTrainData.map(lambda x: x.features[-3]).max()
print parsedTrainData.map(lambda x: x.features[-4]).max()
print parsedTrainData.map(lambda x: x.features[-5]).max()

8761
0.0133333333333
0.0082
0.0217391304348
0.0
0.0

0.797456140351
0.41
0.0869565217391
0.0243902439024
0.25


In [9]:
def squaredError(label, prediction):
    """Calculates the the squared error for a single prediction.

    Args:
        label (float): The correct value for this observation.
        prediction (float): The predicted value for this observation.

    Returns:
        float: The difference between the `label` and `prediction` squared.
    """
    return np.square(label-prediction)

def calcRMSE(labelsAndPreds):
    """Calculates the root mean squared error for an `RDD` of (label, prediction) tuples.

    Args:
        labelsAndPred (RDD of (float, float)): An `RDD` consisting of (label, prediction) tuples.

    Returns:
        float: The square root of the mean of the squared errors.
    """
    return np.sqrt(labelsAndPreds.map(lambda rec: squaredError(rec[0],rec[1])).mean())

In [47]:
averageBike = parsedTrainData.map(lambda lp: lp.label).mean()
print averageBike

191.685766465


In [11]:
labelsAndPredsTrain = parsedTrainData.map(lambda rec: (rec.label, averageBike))
rmseTrainBase = calcRMSE(labelsAndPredsTrain)

labelsAndPredsVal = parsedValData.map(lambda rec: (rec.label, averageBike))
rmseValBase = calcRMSE(labelsAndPredsVal)

print 'Baseline Train RMSE: {:.3f}'.format(rmseTrainBase)
print 'Baseline Validation RMSE: {:.3f}'.format(rmseValBase)

Baseline Train RMSE: 181.530
Baseline Validation RMSE: 180.371


In [ ]:
# Os desafios devem ser obtidos na base de text (parsedTestData)